In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
output_notebook()

from uncertainties import ufloat

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=False, grid=False)

import matplotlib as mpl

# Edit the font, font size, and axes width
# mpl.rcParams['font.family'] = 'Avenir'
plt.rcParams['font.size'] = 24
plt.rcParams['axes.linewidth'] = 2

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/home/trevormjs/Documents/Science/APL/Lab')

from Helper.plotting import my_graph

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
ruby = pd.read_csv('../Lab3/b_ruby.txt', skiprows = 13, delimiter="\t")

no_ruby = pd.read_csv('../Lab3/b_no-ruby.txt', skiprows = 13, delimiter="\t")

ruby.columns = ['Intensity']

no_ruby.columns = ['Intensity']

ruby.insert(0, 'Wavelength(nm)', ruby.index.tolist())

no_ruby.insert(0, 'Wavelength(nm)', ruby.index.tolist())


In [36]:
fig = figure()
fig.line(x=ruby.index, y=ruby.Intensity.values, color='red')
fig.line(x=ruby.index, y=no_ruby.Intensity.values)
show(fig)

In [ ]:
ruby_fft = np.fft.fft(ruby.Intensity.values)
